# AWS Lambda

AWS Lambda is a serverless compute service that runs code in response to events without requiring you to provision or manage servers.

## Key Characteristics

**No servers to manage**: Lambda automatically handles all the infrastructure, including server provisioning, scaling, and maintenance. You only need to upload your code.

**Pay only for execution time**: You are charged based on the number of requests and the compute time consumed. When your code isn't running, you pay nothing.

**Event-driven**: Lambda functions are triggered by events from various AWS services or custom applications. The function executes only when an event occurs.

**Automatically scales**: Lambda automatically scales your application by running code in response to each trigger. Your code runs in parallel and processes each trigger individually.

## Architecture Example

```text
S3 → SNS → SQS (queue)
        ↓
    (10 messages: "angeltech.csv uploaded")
        ↓
    Lambda (checks queue, processes messages)
        ↓
    S3 → CloudWatch Logs

Alternative flow:
Multiple sources (5) → SQS (5 messages) → Lambda
    → Lambda checks:
        1. How many messages in SQS
        2. If SQS = 5 messages
        3. Trigger downstream service
    → EMR Cluster (Airflow DAG for ETL)
```

## Services That Trigger Lambda Functions

Lambda can be triggered by numerous AWS services, including:

**Storage & Data Services**:
- Amazon S3 (object uploads, deletions)
- Amazon DynamoDB (stream changes)

**Messaging & Notifications**:
- Amazon SNS (pub/sub notifications)
- Amazon SQS (queue messages)
- Amazon Simple Email Service (incoming emails)

**Streaming & Real-time Data**:
- Amazon Kinesis Data Firehose
- Amazon Kinesis Data Streams

**API & Integration**:
- Amazon API Gateway (HTTP requests)
- AWS IoT Button
- Amazon CloudWatch (scheduled events, alarms)
- AWS CloudFormation
- Amazon Lex (chatbot interactions)
- Amazon Cognito (user authentication)
- Amazon CloudFront (edge computing)
- AWS CodeCommit (code repository changes)
- AWS Config (configuration changes)

# Lambda Costs

## How AWS Lambda Charges

Lambda uses a pay-per-use pricing model with three main components:

**Pay per request**: Charged by the number of function invocations (how many times your function is executed).

**Pay per execution time**: Billed by milliseconds, measured from when your code begins executing until it returns or terminates.

**Pay per memory allocation**: CPU power scales proportionally with memory size. More memory means more CPU power but higher cost.

## Key Cost Factors

**Invocation count**: Total number of times your Lambda function is executed.

**Execution duration**: How long your function runs (billed in milliseconds).

**Memory configuration**: The amount of memory allocated to your function (128 MB to 10,240 MB).

## Important Cost Notes

- **No cost when Lambda is not running**: You only pay for actual execution time.
- **Short, fast executions are cheaper**: Optimizing your code to run faster reduces costs.
- **High-frequency triggers can become expensive**: If your Lambda is triggered millions of times, costs add up despite the low per-request charge.

## Pricing Details

**Generous Free Tier (per month)**:
- 1 million requests
- 400,000 GB-seconds of compute time

**After Free Tier**:
- $0.20 per 1 million requests
- $0.00001667 per GB-second

## GB-Second Calculation

GB-second is calculated as: Memory (in GB) × Execution time (in seconds)

**Examples**:

Example 1:
```text
Lambda configuration: 1GB memory, 1 second execution
GB-seconds = 1 × 1 = 1 GB-second
```

Example 2:
```text
Lambda configuration: 2GB memory, 3 seconds execution
GB-seconds = 2 × 3 = 6 GB-seconds
```

Example 3 (Cost calculation for 1 million invocations):
```text
Configuration: 1GB memory, 2 seconds execution
Per invocation: 1 × 2 = 2 GB-seconds
For 1,000,000 invocations: 2 × 1,000,000 = 2,000,000 GB-seconds
Cost: 2,000,000 × $0.00001667 = $33.34
For 30 days: $33.34 × 30 = $1,000.20
```

Practice problem:
```text
Configuration: 512MB memory (0.512 GB), 10 seconds execution
GB-seconds per invocation = 0.512 × 10 = 5.12 GB-seconds
```

# Creating a Lambda Function

## Step 1: Choose Creation Method

**Author from scratch**: Start with a blank function and write your own code.

**Use a blueprint**: Start from a pre-built template for common use cases (e.g., S3 object processing, DynamoDB streams).

**Container image**: Deploy your function as a container image (up to 10 GB), useful for complex dependencies.

## Step 2: Basic Configuration

**Function name**: Give your Lambda function a unique, descriptive name.

**Runtime**: Choose the programming language and version:
- Python (3.8, 3.9, 3.10, 3.11, etc.)
- Node.js
- Java
- Go
- Ruby
- .NET Core
- Custom runtime

**Architecture**: Select the instruction set architecture:
- x86_64 (traditional Intel/AMD)
- arm64 (AWS Graviton2, often cheaper and more efficient)

## Step 3: Permissions

**Assign execution role**: Lambda needs an IAM role to execute and access other AWS services.

**Grant access to AWS services**: The role must have permissions to interact with services like:
- S3 (read/write objects)
- SQS (receive/delete messages)
- DynamoDB (read/write items)
- CloudWatch Logs (write logs)

Common managed policies:
- `AWSLambdaBasicExecutionRole` (CloudWatch Logs access)
- `AWSLambdaVPCAccessExecutionRole` (VPC access)
- Custom policies for S3, DynamoDB, SQS, etc.

## Step 4: Advanced Options

**Durable execution (optional)**: Configure for long-running workflows.

**Environment variables**: Store configuration values (API keys, database URLs) as key-value pairs accessible in your code.

**Timeout settings**: Maximum execution time (default: 3 seconds, max: 15 minutes/900 seconds).

**Memory settings**: Allocate memory from 128 MB to 10,240 MB (CPU scales proportionally).

Additional advanced settings:
- VPC configuration (for accessing resources in a private network)
- File system mounting (EFS)
- Concurrency limits
- Dead letter queue configuration
- Layers (reusable code/dependencies)

# Amazon SQS (Simple Queue Service)

## What is SQS?

Amazon SQS is a fully managed message queue service that enables you to decouple and scale microservices, distributed systems, and serverless applications.

**Fully managed**: AWS handles all infrastructure, maintenance, and scaling.

**Decouples producers and consumers**: Producers send messages to the queue, consumers retrieve and process them independently. They don't need to communicate directly.

## Why Use SQS?

**Buffer traffic spikes**: Queue absorbs sudden bursts of requests, preventing downstream systems from being overwhelmed.

**Improve system reliability**: If a consumer fails, messages remain in the queue and can be processed later.

**Enable asynchronous processing**: Producers don't wait for consumers to finish processing; they continue working independently.

## Core Concepts

**Message**: A unit of data sent to the queue (up to 256 KB). Can contain text, JSON, XML, or any unformatted text.

**Queue**: A temporary repository for messages waiting to be processed.

**Producer**: The component or service that sends messages to the queue.

**Consumer**: The component or service that retrieves and processes messages from the queue.

**Visibility Timeout**: When a consumer receives a message, it becomes invisible to other consumers for a specified period (default: 30 seconds). This prevents multiple consumers from processing the same message. If the consumer doesn't delete the message within this timeout, it becomes visible again for reprocessing.

## Queue Types

### Standard Queue (Default)

- Unlimited throughput
- Best-effort ordering (messages might arrive out of order)
- At-least-once delivery (messages might be delivered more than once)
- Available in all AWS regions

### FIFO Queue (First In, First Out)

- **First in, first out ordering**: Messages are processed in the exact order they're sent.
- **Not available in all regions**: Check regional availability.
- **Queue name must end with .fifo**: This is a naming requirement (e.g., `my-queue.fifo`).
- **Exactly-once processing**: Messages are delivered exactly once and remain available until a consumer processes and deletes them.
- **Lower throughput**: Up to 3,000 messages per second with batching (or 300 messages per second without batching).

## SQS Limitations

**Maximum in-flight messages**: 120,000 messages can be in-flight (received by consumers but not yet deleted) at any time.

**Batch request maximum**: 10 messages per batch request, with a total maximum size of 256 KB.

**Message content types**: XML, JSON, or unformatted text.

**FIFO throughput**: Supports up to 3,000 messages per second using batching.

**Maximum message size**: 256 KB per message. For larger payloads, use S3 and send a reference in the message.

**Data retention**: Messages can be retained from 1 minute to 14 days (default: 4 days).

**Pricing**: Calculated per API request (send, receive, delete) and network data transfer usage.

## Common Use Cases

- Decoupling microservices
- Work queues for distributed systems
- Buffering writes to databases
- Batch processing
- Order processing systems
- Fan-out pattern with SNS

# Amazon SNS (Simple Notification Service)

## What is SNS?

Amazon SNS is a fully managed publish-subscribe (pub/sub) messaging service that enables you to send messages to multiple subscribers simultaneously.

**Fully managed pub/sub service**: AWS handles all infrastructure and message delivery.

**One message → fan-out to multiple subscribers**: A single published message can be delivered to many different endpoints.

**Decouples message producers from consumers**: Publishers don't need to know who the subscribers are or how they process messages.

## How SNS Works

1. Create a **topic** (a logical access point and communication channel)
2. **Subscribers** subscribe to the topic
3. **Publishers** send messages to the topic
4. SNS delivers the message to all subscribers

## Common Subscribers

SNS can deliver messages to various endpoints:

**Email / SMS**: Send notifications to email addresses or mobile phone numbers.

**SQS**: Send messages to SQS queues for further processing.

**Lambda**: Trigger Lambda functions to execute code in response to messages.

**HTTP / HTTPS endpoints**: Send messages to web servers or APIs.

**Mobile push notifications**: Send to mobile apps (iOS, Android, etc.).

**Application endpoints**: Custom application endpoints.

## Key Features

**Fan-out pattern**: Publish once, deliver to many subscribers (common pattern: SNS → multiple SQS queues).

**Message filtering**: Subscribers can filter messages based on message attributes.

**Message attributes**: Add metadata to messages for routing and filtering.

**Delivery retry policies**: Automatic retries for failed deliveries.

**Dead-letter queues**: Send failed messages to an SQS queue for analysis.

## Common Architecture Pattern: SNS + SQS

```text
Producer → SNS Topic → Multiple SQS Queues → Multiple Lambda Functions
```

Benefits:
- Publish once to SNS
- Multiple independent processing pipelines via SQS
- Each queue can have different consumers
- Fault tolerance (messages persist in SQS)
- Independent scaling per consumer

# S3 Event-Driven Architecture

## What are S3 Events?

S3 can emit events when objects are created, modified, or deleted in a bucket. These events can trigger downstream processing in near real-time.

## Common S3 Event Types

**ObjectCreated events**:
- PUT: Object uploaded via PUT request
- POST: Object uploaded via POST request
- COPY: Object created via copy operation
- CompleteMultipartUpload: Large object uploaded in parts

**ObjectRemoved events**:
- DELETE: Object deleted from bucket
- DeleteMarkerCreated: Delete marker added (versioned bucket)

**Other event types**:
- ObjectRestore: Object restored from Glacier
- Replication events: Cross-region replication completed
- Object tagging events

## Event Destinations

S3 events can trigger various AWS services:

**AWS Lambda**: Execute code in response to S3 changes (most common pattern).

**Amazon SQS**: Send event notifications to a queue for asynchronous processing.

**Amazon SNS**: Publish event notifications to multiple subscribers.

**EventBridge**: Route events to multiple targets with advanced filtering.

## Architecture Diagram

```text
                    ┌─────────────┐
                    │  SNS Topic  │
                    └─────────────┘
                          ▲
                          │
                    ┌─────────────┐        ┌──────────────────┐
                    │  S3 Bucket  │───────▶│  Router Lambda   │
                    └─────────────┘        └──────────────────┘
                          │
                          │
                    ┌─────────────┐
                    │  SQS Queue  │
                    └─────────────┘
```

## Typical Use Cases

**Trigger data pipelines**: When new data files arrive, automatically start ETL processes.

**File-driven ETL**: Process CSV, JSON, or Parquet files as soon as they're uploaded.

**Near real-time processing**: Respond to data changes within seconds.

**Image/video processing**: Automatically resize images or transcode videos upon upload.

**Data validation**: Validate file formats and data quality immediately after upload.

**Archival and backup**: Copy files to Glacier or another region automatically.

**Audit logging**: Track all object changes for compliance.

## Example Flow: S3 → SNS → SQS → Lambda

```text
1. File uploaded to S3 bucket
2. S3 sends event notification to SNS topic
3. SNS fans out to multiple SQS queues
4. Lambda functions poll SQS queues
5. Lambda processes file from S3
6. Lambda writes results back to S3
7. CloudWatch logs capture execution details
```

**Benefits of this pattern**:
- Decoupling: Components work independently
- Buffering: SQS absorbs traffic spikes
- Retry logic: Failed messages can be reprocessed
- Parallel processing: Multiple Lambdas process different messages
- Fan-out: Single S3 event triggers multiple workflows

## S3 Event Configuration

To set up S3 events:

1. Go to S3 bucket properties
2. Navigate to "Event notifications"
3. Create event notification
4. Choose event types (ObjectCreated, ObjectRemoved, etc.)
5. Add prefix/suffix filters (optional, e.g., only `.csv` files)
6. Select destination (Lambda, SQS, SNS)
7. Configure destination permissions (Lambda execution role needs S3 read access)

# Amazon CloudWatch

## What is CloudWatch?

Amazon CloudWatch is a monitoring and observability service that provides data and actionable insights for AWS resources and applications.

**Monitoring service**: Tracks the health and performance of your AWS infrastructure.

**Observability service**: Provides visibility into logs, metrics, and traces across your entire stack.

## Core Capabilities

**Collects and tracks metrics**: Monitors CPU utilization, network traffic, disk I/O, custom application metrics.

**Collects, stores, and searches log files**: Centralized log management from Lambda, EC2, ECS, and other services.

**Enables alarms and automated actions**: Trigger notifications or automated responses when thresholds are breached.

**Provides visibility**: Unified view of resource health, application performance, and operational status.

## Typical CloudWatch Workflow

```text
1. AWS resources emit metrics & logs
   ↓
2. CloudWatch collects and stores data
   ↓
3. CloudWatch evaluates thresholds and conditions
   ↓
4. Alarms trigger when thresholds are breached
   ↓
5. Actions are executed:
   - Send email via SNS
   - Scale resources via Auto Scaling
   - Trigger Lambda function
   - Post to SNS topic
```

## CloudWatch Components

### Metrics
- Data points about resource performance
- Examples: CPUUtilization, NetworkIn, DiskReadOps
- Can create custom metrics
- Stored for 15 months

### Logs
- Log groups (collection of log streams)
- Log streams (sequence of log events from same source)
- Log insights (query and analyze logs)
- Integration with Lambda, EC2, ECS, etc.

### Alarms
- Monitor a single metric
- Execute actions when thresholds are breached
- States: OK, ALARM, INSUFFICIENT_DATA
- Can trigger SNS, Auto Scaling, EC2 actions

### Dashboards
- Customizable views of metrics
- Multiple widgets showing different data
- Shareable across accounts

## CloudWatch in Lambda Context

**Automatic log integration**: Every Lambda function automatically sends logs to CloudWatch Logs.

**Log groups**: Created automatically with format `/aws/lambda/<function-name>`.

**Log streams**: Each Lambda invocation creates a log entry in a stream.

**Monitoring Lambda metrics**:
- Invocations: Number of times function is executed
- Duration: Execution time
- Errors: Number of failed invocations
- Throttles: Number of throttled invocations
- Concurrent executions: Number of function instances running simultaneously
- Dead letter queue errors: Messages failed to process

**CloudWatch Logs Insights**: Query Lambda logs for debugging and analysis.

## Architecture Diagram

```text
                       ┌─────────────────────────────────────┐
                       │      Amazon CloudWatch              │
                       │                                     │
   ┌──────────────┐    │  ┌──────────┐   ┌──────────────┐  │    ┌─────────┐
   │ AWS Resources│───▶│  │ Metrics  │   │  Statistics  │  │───▶│ Actions │
   │ (CloudWatch  │    │  └──────────┘   └──────────────┘  │    └─────────┘
   │   enabled)   │    │                                     │         │
   └──────────────┘    │  ┌──────────────────────────────┐ │         ├──▶ Email
                       │  │  Alarms                      │ │         │     (SNS)
   ┌──────────────┐    │  │  - CPU > 80%                │ │         │
   │ Custom Data  │───▶│  │  - Memory > 90%             │ │         └──▶ Auto
   │              │    │  │  - Errors > 5               │ │               Scaling
   └──────────────┘    │  └──────────────────────────────┘ │
                       │                                     │
                       └─────────────────────────────────────┘
                                         │
                                         ▼
                         ┌─────────────────────────────┐
                         │  AWS Management Console     │
                         │  Statistics Consumer        │
                         └─────────────────────────────┘
```

